In [ ]:
# DB from https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset
# By Stefano Leone - https://www.kaggle.com/stefanoleone992
# Published under CC0 license.

In [ ]:
import gzip
import os
import pathlib
import sqlite3
from typing import Optional, Tuple

import numpy as np
import pandas as pd

In [ ]:
from sqlalchemy import (
    Column, DateTime, Float, ForeignKey,
    Integer, String, Table, create_engine,
)
from sqlalchemy.ext.declarative import declarative_base

DB_PATH = 'imdb3.db'

engine = create_engine(f'sqlite:///{DB_PATH}', echo=True)
Base = declarative_base()

In [ ]:
class Country(Base):
    __tablename__ = 'countries'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)


class Genre(Base):
    __tablename__ = 'genres'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)


class Language(Base):
    __tablename__ = 'languages'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)


movie_country = Table('movie_countries', Base.metadata,
    Column('movie_id', String, ForeignKey('movies.id')),
    Column('country_id', Integer, ForeignKey('countries.id'))
)


movie_genre = Table('movie_genres', Base.metadata,
    Column('movie_id', String, ForeignKey('movies.id')),
    Column('genre_id', Integer, ForeignKey('genres.id'))
)


movie_language = Table('movie_languages', Base.metadata,
    Column('movie_id', String, ForeignKey('movies.id')),
    Column('language_id', Integer, ForeignKey('languages.id'))
)

    
class Movie(Base):
    __tablename__ = 'movies'

    id = Column(String, primary_key=True)
    title = Column(String, nullable=False)
    original_title = Column(String, nullable=False)
    year = Column(Integer, nullable=False)
    avg_vote = Column(String, nullable=False)
    votes = Column(Integer, nullable=False)
    duration = Column(Integer, nullable=False)
    budget = Column(String)
    gross_income = Column(String)

    
class Name(Base):
    __tablename__ = 'names'

    id = Column(String, primary_key=True)
    name = Column(String, nullable=False)
    height = Column(Float)
    date_of_birth = Column(DateTime, nullable=True)
    date_of_death = Column(DateTime, nullable=True)
    children = Column(Integer, nullable=False)


class Principal(Base):
    __tablename__ = 'principals'

    movie_id = Column(String, ForeignKey('movies.id'), primary_key=True)
    ordering = Column(String, primary_key=True)
    name_id = Column(String, ForeignKey('names.id'), nullable=False)
    job_id = Column(Integer, ForeignKey('jobs.id'), nullable=False)
    characters = Column(String)


class Job(Base):
    __tablename__ = 'jobs'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
CREATE_TABLE_SETTINGS = {'con': engine, 'if_exists': 'append'}

In [ ]:
names = pd.read_csv(
    'names.csv',
    parse_dates=['date_of_birth', 'date_of_death'],
    infer_datetime_format=True,
    na_values="None",
)[
    ['imdb_name_id', 'name', 'height',
     'date_of_birth', 'date_of_death', 'children']
]

principals = pd.read_csv(
    'title_principals.csv', na_values="None",
)[['imdb_title_id', 'ordering', 'imdb_name_id', 'category', 'characters']]

movies = pd.read_csv(
    'movies.csv', parse_dates=['date_published'], na_values="None",
)[
    ['imdb_title_id', 'title', 'original_title', 'year', 'genre',
     'duration', 'avg_vote', 'votes', 'country', 'language', 'budget',
     'worlwide_gross_income']
]


movies.rename(
    columns={'imdb_title_id': 'id', 'worlwide_gross_income': 'gross_income'},
    inplace=True,
)
principals.rename(
    columns={'imdb_title_id': 'movie_id', 'imdb_name_id': 'name_id',
             'category': 'job_id'},
    inplace=True,
)
names.rename(columns={'imdb_name_id': 'id'}, inplace=True)

In [ ]:
names['date_of_birth'] = pd.to_datetime(names['date_of_birth'], errors='coerce')
names['date_of_death'] = pd.to_datetime(names['date_of_death'], errors='coerce')

In [ ]:
movies.set_index('id', inplace=True)
names.set_index('id', inplace=True)
principals.set_index(['movie_id', 'ordering'], inplace=True)

In [ ]:
tables = {
    'names': names,
    'principals': principals,
    'movies': movies,
}

In [ ]:
# Create M2Ms

TO_CONVERT = [
    ('movies', 'genre', 'genres'),
    ('movies', 'language', 'languages'),
    ('movies', 'country', 'countries'),
]

for t, c, many in TO_CONVERT:
    values = {
        value.strip()
        for vals in tables[t][c].str.split(',')
        for value in (vals if isinstance(vals, list) else [])
    }

    second_table = list(enumerate(values, 1))
    tables[many] = pd.DataFrame(second_table, columns=['id', 'name'])
    tables[many].set_index('id', inplace=True)

    # Create the M2M relationships
    values_kv = {v: k for k, v in second_table}
    titles_value = (
        (i, values_kv.get(value))
        for i, title in tables[t].iterrows() if isinstance(title[c], str)
        for value in map(str.strip, title[c].split(','))
    )

    tables[f'movie_{many}'] = pd.DataFrame(titles_value, columns=['movie_id', f'{c}_id'])
    tables[t].drop([c], axis=1, inplace=True)

In [ ]:
jobs = list(enumerate(principals['job_id'].unique(), 1))
tables['jobs'] = pd.DataFrame(jobs, columns=['id', 'name'])
for job_id, job_name in jobs:
    tables['principals'].replace({job_name: job_id}, inplace=True)

In [ ]:
delete_chars = str.maketrans("", "", "[]\"")

tables['principals']['characters'] = (
    tables['principals']['characters'].astype(str)
    .str.translate(delete_chars)
    .replace(',', ', ').replace('nan', np.nan)
)

In [ ]:
WITHOUT_INDEX = {'movie_genres', 'movie_languages', 'movie_countries', 'jobs'}

for name, df in tables.items():
    CREATE_TABLE_SETTINGS['index'] = name not in WITHOUT_INDEX
    print(name, CREATE_TABLE_SETTINGS)
    df.to_sql(name, **CREATE_TABLE_SETTINGS)